In [1]:
cd Desktop

/Users/parisamofakham/Desktop


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.

In [3]:
cd /Users/parisamofakham/Desktop

/Users/parisamofakham/Desktop


In [4]:
#data preprocessing inspired by Bojan

#basic packages
import numpy as np 
import pandas as pd 
import os
import gc

#natural language processing in Russian
#import nltk
#nltk.download()

#term frequency
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords 

#other
from sklearn import preprocessing

In [5]:
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4



/Users/parisamofakham/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/parisamofakham/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
#import raw data#import 
training = pd.read_csv('train.csv', index_col = "item_id", parse_dates = ["activation_date"])
testing = pd.read_csv('test.csv', index_col = "item_id", parse_dates = ["activation_date"])
traindex = training.index
testdex = testing.index

In [8]:
#feature engineering:price (training)
training["price"] = np.log(training["price"]+0.001)
training["price"].fillna(-999,inplace=True)
training["image_top_1"].fillna(-999,inplace=True)

training["Weekday"] = training['activation_date'].dt.weekday
training["Weekd of Year"] = training['activation_date'].dt.week
training["Day of Month"] = training['activation_date'].dt.day

#feature engineerin:price (testing)
testing["price"] = np.log(testing["price"]+0.001)
testing["price"].fillna(-999,inplace=True)
testing["image_top_1"].fillna(-999,inplace=True)

testing["Weekday"] = testing['activation_date'].dt.weekday
testing["Weekd of Year"] = testing['activation_date'].dt.week
testing["Day of Month"] = testing['activation_date'].dt.day

In [9]:
#create validation index, remove variables we won't use, and encode remaining variables
#training
training_index = training.loc[training.activation_date<=pd.to_datetime('2017-04-07')].index
train_val_index = training.loc[training.activation_date>=pd.to_datetime('2017-04-08')].index
training.drop(["activation_date","image"],axis=1,inplace=True)

categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1"]
lbl = preprocessing.LabelEncoder()
for col in categorical:
    training[col] = lbl.fit_transform(training[col].astype(str))

#testing
testing_index = testing.loc[testing.activation_date<=pd.to_datetime('2017-04-07')].index
test_val_index = testing.loc[testing.activation_date>=pd.to_datetime('2017-04-08')].index
testing.drop(["activation_date","image"],axis=1,inplace=True)

#categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1"] already defined above
lbl = preprocessing.LabelEncoder()
for col in categorical:
    testing[col] = lbl.fit_transform(testing[col].astype(str))

In [10]:
#feature engineering: text (training)
training['text_feat'] = training.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
training.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

#feature engineering: text (testing)
testing['text_feat'] = testing.apply(lambda row: ' '.join([
    str(row['param_1']), 
    str(row['param_2']), 
    str(row['param_3'])]),axis=1) # Group Param Features
testing.drop(["param_1","param_2","param_3"],axis=1,inplace=True)

In [11]:
#meta text features (training)
textfeats = ["description","text_feat", "title"]
for cols in textfeats:
    training[cols] = training[cols].astype(str) 
    training[cols] = training[cols].astype(str).fillna('nicapotato') # FILL NA
    training[cols] = training[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    training[cols + '_num_chars'] = training[cols].apply(len) # Count number of Characters
    training[cols + '_num_words'] = training[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    training[cols + '_num_unique_words'] = training[cols].apply(lambda comment: len(set(w for w in comment.split())))
    training[cols + '_words_vs_unique'] = training[cols+'_num_unique_words'] / training[cols+'_num_words'] * 100 # Count Unique Words
    
#meta text features (testing)
#textfeats = ["description","text_feat", "title"], already defined above
for cols in textfeats:
    testing[cols] = testing[cols].astype(str) 
    testing[cols] = testing[cols].astype(str).fillna('nicapotato') # FILL NA
    testing[cols] = testing[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently
    testing[cols + '_num_chars'] = testing[cols].apply(len) # Count number of Characters
    testing[cols + '_num_words'] = testing[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    testing[cols + '_num_unique_words'] = testing[cols].apply(lambda comment: len(set(w for w in comment.split())))
    testing[cols + '_words_vs_unique'] = testing[cols+'_num_unique_words'] / testing[cols+'_num_words'] * 100 # Count Unique Words

In [12]:
testing.head()

,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,user_type,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
item_id,,,,,,,,,,,,,,,,,,,,,
6544e41a8817,262938,4,306,4,10,отдам бесплатно,на ангарском,-999.000000,66,1,...,2,100.0,22,4,4,100.000000,15,2,2,100.0
65b9484d670f,55145,19,933,8,5,продам велосипед,"продам велосипед kama f200,в нормальном состо...",8.006368,4,1,...,10,100.0,16,3,2,66.666667,16,2,2,100.0
8bab230b2ecd,13649,12,147,0,2,bbk,продам новый телевизор bbk 32 диагональ смарт...,9.615806,15,1,...,21,100.0,30,5,4,80.000000,3,1,1,100.0
8e348601fefc,113959,18,1238,2,4,вытяжка jetair 60,"продам новую вытяжку в упаковке,с документами....",8.411833,70,1,...,12,100.0,21,4,4,100.000000,17,3,3,100.0
8bd2fe400b89,42880,14,236,4,42,коляска зима-лето,продам отличную коляску. б/у 1 год. все вопрос...,8.496991,15,1,...,10,100.0,23,4,3,75.000000,17,2,2,100.0


In [13]:
#set stopwords
russian_stop = set(stopwords.words('russian'))

#term frequency parameters
tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}

#define function to create term frequency variables
def get_col(col_name): return lambda x: x[col_name]

#create vectorized NLP variables
vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            #max_features=7000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            #max_features=7000,
            preprocessor=get_col('title')))
    ])

In [14]:
y = training.deal_probability.copy()

In [15]:
training.drop("deal_probability",axis=1, inplace=True)
print('Train shape: {} Rows, {} Columns'.format(*training.shape))


Train shape: 1503424 Rows, 27 Columns


In [16]:
training.head()

,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,user_type,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,675853,19,460,4,42,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",5.991467,2,1,...,7,100.0,33,4,3,75.000000,21,3,3,100.0
2dac0150717d,173962,17,1300,2,22,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",8.006368,19,1,...,7,100.0,14,3,2,66.666667,17,3,3,100.0
ba83aefab5dc,440069,16,1276,0,2,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",8.294050,9,1,...,17,100.0,35,7,6,85.714286,14,2,2,100.0
02996f1dd2ea,576929,21,940,4,42,автокресло,продам кресло от0-25кг,7.696213,286,0,...,3,100.0,28,4,3,75.000000,10,1,1,100.0
7c90be56d2ab,721853,4,317,6,0,"ваз 2110, 2003",все вопросы по телефону.,10.596635,3,1,...,4,100.0,26,5,5,100.000000,14,3,3,100.0


In [17]:
print("\nEncode Variables")
categorical = ["user_id","region","city","parent_category_name","category_name","user_type","image_top_1"]
print("Encoding :",categorical)




Encode Variables
Encoding : ['user_id', 'region', 'city', 'parent_category_name', 'category_name', 'user_type', 'image_top_1']


In [18]:
#Importing Relevant Libraries
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [19]:
training.head()

,user_id,region,city,parent_category_name,category_name,title,description,price,item_seq_number,user_type,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,675853,19,460,4,42,кокоби(кокон для сна),"кокон для сна малыша,пользовались меньше месяц...",5.991467,2,1,...,7,100.0,33,4,3,75.000000,21,3,3,100.0
2dac0150717d,173962,17,1300,2,22,стойка для одежды,"стойка для одежды, под вешалки. с бутика.",8.006368,19,1,...,7,100.0,14,3,2,66.666667,17,3,3,100.0
ba83aefab5dc,440069,16,1276,0,2,philips bluray,"в хорошем состоянии, домашний кинотеатр с blu ...",8.294050,9,1,...,17,100.0,35,7,6,85.714286,14,2,2,100.0
02996f1dd2ea,576929,21,940,4,42,автокресло,продам кресло от0-25кг,7.696213,286,0,...,3,100.0,28,4,3,75.000000,10,1,1,100.0
7c90be56d2ab,721853,4,317,6,0,"ваз 2110, 2003",все вопросы по телефону.,10.596635,3,1,...,4,100.0,26,5,5,100.000000,14,3,3,100.0


In [20]:
#Dropping the non-numeric variables
training.drop("title", axis=1, inplace=True)
training.drop("description", axis=1, inplace=True)
training.drop("text_feat", axis=1, inplace= True)

In [23]:
training.head(2)

,user_id,region,city,parent_category_name,category_name,price,item_seq_number,user_type,image_top_1,Weekday,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
item_id,,,,,,,,,,,,,,,,,,,,,
b912c3c6a6ad,675853,19,460,4,42,5.991467,2,1,13,1,...,7,100.0,33,4,3,75.000000,21,3,3,100.0
2dac0150717d,173962,17,1300,2,22,8.006368,19,1,2722,6,...,7,100.0,14,3,2,66.666667,17,3,3,100.0


In [24]:
print(training.columns)

Index(['user_id', 'region', 'city', 'parent_category_name', 'category_name',
       'price', 'item_seq_number', 'user_type', 'image_top_1', 'Weekday',
       'Weekd of Year', 'Day of Month', 'description_num_chars',
       'description_num_words', 'description_num_unique_words',
       'description_words_vs_unique', 'text_feat_num_chars',
       'text_feat_num_words', 'text_feat_num_unique_words',
       'text_feat_words_vs_unique', 'title_num_chars', 'title_num_words',
       'title_num_unique_words', 'title_words_vs_unique'],
      dtype='object')


In [27]:
"""
Using Randomized train/valid split doesn't seem to generalize LB score, so I will try time cutoff
"""
X_train, X_valid, y_train, y_valid = train_test_split(
    training, y, test_size=0.20, random_state=23)

In [28]:
#Importing Relevant Libraries
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 100, 'max_depth': 6,
        'learning_rate': 0.1, 'loss': 'huber','alpha':0.95}


In [29]:
#Gradient Boosting Modeling
clf = GradientBoostingRegressor(**params).fit(X_train, y_train)


In [30]:
#Error Terms Based on Validation Data
mse = mean_squared_error(y_valid, clf.predict(X_valid))
r2 = r2_score(y_valid, clf.predict(X_valid))

print("MSE: %.4f" % mse)
print("R2: %.4f" % r2)


MSE: 0.0538
R2: 0.2058


In [130]:
# Defining RMSE Function
def rmse(p,t):
    return np.sqrt(((p-t)**2).mean())

In [138]:
#RMSE of the Gradient Boosting Model on the Validation Data
rmse(clf.predict(X_valid),y_valid)

0.231865305612452

In [75]:
training.reset_index()


,item_id,user_id,region,city,parent_category_name,category_name,price,item_seq_number,user_type,image_top_1,...,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
0,b912c3c6a6ad,675853,19,460,4,42,5.991467,2,1,13,...,7,100.000000,33,4,3,75.000000,21,3,3,100.000000
1,2dac0150717d,173962,17,1300,2,22,8.006368,19,1,2722,...,7,100.000000,14,3,2,66.666667,17,3,3,100.000000
2,ba83aefab5dc,440069,16,1276,0,2,8.294050,9,1,2260,...,17,100.000000,35,7,6,85.714286,14,2,2,100.000000
3,02996f1dd2ea,576929,21,940,4,42,7.696213,286,0,2837,...,3,100.000000,28,4,3,75.000000,10,1,1,100.000000
4,7c90be56d2ab,721853,4,317,6,0,10.596635,3,1,1408,...,4,100.000000,26,5,5,100.000000,14,3,3,100.000000
5,51e0962387f7,566683,21,1657,4,42,7.170120,9,1,2837,...,3,100.000000,28,4,3,75.000000,11,2,2,100.000000
6,c4f260a2b48a,27020,11,975,2,38,9.305651,125,1,2027,...,19,90.476190,26,5,4,80.000000,45,5,5,100.000000
7,6b71309d6a8a,768696,15,1153,4,29,6.214610,61,1,2584,...,4,100.000000,24,4,4,100.000000,16,2,2,100.000000
8,c5b969cb63a2,16061,14,1107,4,29,6.214610,85,1,2416,...,5,100.000000,39,8,8,100.000000,6,1,1,100.000000
9,b1570962e68c,753379,11,975,4,10,5.991467,136,0,2465,...,11,100.000000,20,4,4,100.000000,33,4,4,100.000000


In [77]:
#Saving the Training Prediction into a Dataframe
x1= pd.DataFrame(clf.predict(X_train))

In [82]:
# Saving Actucal Train Probability on a Dataframe
ytrain = pd.DataFrame(y_train)

In [85]:
ytrain.reset_index()

,item_id,deal_probability
0,a4697bfc834c,0.00000
1,90e4b4336367,0.00000
2,05cfa54bd6fe,0.00000
3,6ca2b9c77f18,0.00000
4,d637ba77e8ab,0.20000
5,c86aeb1f92a7,0.00000
6,3dd6508078e0,0.00000
7,7ce45598b909,0.00000
8,45bb4a42723b,0.00000
9,e943087403d3,0.00000


### Adding Item_Id to the  Training Prediction and Actual Probability Datastes 

In [93]:
item_id1 = X_train.index

In [94]:
item_id2 = ytrain.index

In [95]:
X_train['item_id'] = item_id1

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
ytrain['item_id'] = item_id2

In [97]:
#Merging Actual Probability and the Prediction Datasets on Their Item_Ids
finaltrain = pd.merge(ytrain, X_train, on= 'item_id')



In [139]:
finaltrain.head

<bound method NDFrame.head of          deal_probability       item_id  user_id  region  city  \
0                 0.00000  a4697bfc834c   159805       9     3   
1                 0.00000  90e4b4336367   720329       8   645   
2                 0.00000  05cfa54bd6fe    59637       1  1589   
3                 0.00000  6ca2b9c77f18     2552       3   672   
4                 0.20000  d637ba77e8ab   147860      25  1441   
5                 0.00000  c86aeb1f92a7   254781      12  1048   
6                 0.00000  3dd6508078e0   203860      15  1123   
7                 0.00000  7ce45598b909   541458       3    31   
8                 0.00000  45bb4a42723b     4330      10   743   
9                 0.00000  e943087403d3     8269      25  1441   
10                0.21159  54ca73f55ea5   325834      14  1107   
11                0.57143  132c016daa9f    66905       1  1589   
12                0.00000  c05a8c661e9c   704021      26  1632   
13                0.00000  c21fc8e2bf31   1750

In [103]:
#Subsetting finaltrain to only include item_id and deal_probability
finaltrain1 = finaltrain[['item_id', 'deal_probability']]

In [104]:
#Adding the prediction as a new column to finaltrain1
finaltrain1['prediction'] = x1

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [140]:
finaltrain1.head

<bound method NDFrame.head of               item_id  deal_probability  prediction
0        a4697bfc834c           0.00000    0.253415
1        90e4b4336367           0.00000    0.041180
2        05cfa54bd6fe           0.00000    0.073421
3        6ca2b9c77f18           0.00000    0.080723
4        d637ba77e8ab           0.20000    0.371598
5        c86aeb1f92a7           0.00000    0.061039
6        3dd6508078e0           0.00000    0.243032
7        7ce45598b909           0.00000    0.054268
8        45bb4a42723b           0.00000    0.040957
9        e943087403d3           0.00000    0.054800
10       54ca73f55ea5           0.21159    0.185694
11       132c016daa9f           0.57143    0.297885
12       c05a8c661e9c           0.00000    0.057706
13       c21fc8e2bf31           0.00000    0.087261
14       55942d28833f           0.73760    0.357985
15       a36335c3e19b           0.00000    0.211873
16       da27dab1b3cb           0.00000    0.026600
17       c519e6328e4a           0.

In [141]:
#Saving the validation prediction into a Dataframe
x2= pd.DataFrame(clf.predict(X_valid))

In [106]:
#Saving the Validation Prediction into a Dataframe
yvalid = pd.DataFrame(y_valid)

### Adding Item_Id to the  Validation Prediction and Actual Probability Datastes 

In [112]:
item_id2 = X_valid.index

In [114]:
item_id3 = yvalid.index

In [113]:
X_valid['item_id'] = item_id2

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [115]:
yvalid['item_id'] = item_id3

In [116]:
#Merging Validation Prediction and Actual Probabilities
finalvalid = pd.merge(yvalid, X_valid, on= 'item_id')



In [117]:
#Subsetting finalVALID to only include item_id and deal_probability
finalvalid1 = finalvalid[['item_id', 'deal_probability']]

In [120]:
# adding the prediction as a colum to the finalvalid data
finalvalid1['prediction'] = x2

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
finalvalid1.head()

,item_id,deal_probability,prediction
0,6a46d8357f52,0.00000,0.031442
1,d962ff71addc,0.00000,0.132996
2,ee52490e29b3,0.21301,0.103664
3,dab8b2a9af06,0.00000,0.062797
4,64c264dc28e5,0.00000,0.049875


In [126]:
# Concatinate finaltrain1 and finalvalid1
df = pd.concat([finaltrain1, finalvalid1])



In [128]:
df.shape

(1503424, 3)

In [129]:
#Save the training data into a csv file
df.to_csv('xgbtrain.csv', index=False)

# Now Moving on to the Test Data

In [60]:
# Deleting the Non-numeric Variables
testing.drop("title", axis=1, inplace=True)
testing.drop("description", axis=1, inplace=True)
testing.drop("text_feat", axis=1, inplace= True)

In [64]:
# Implementing Our Prediction Model to the Test data and creating a deal_prbability column
testing["deal_probability"] = clf.predict(testing)

In [67]:
item_id = testing.index

In [69]:
#defining a Dataframe which only includes deal_probability
submission = testing[['deal_probability']]

In [70]:
#Adding item_id to the submission file
submission['item_id'] = item_id

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [71]:
submission.head()

,deal_probability,item_id
item_id,,
6544e41a8817,0.053024,6544e41a8817
65b9484d670f,0.271194,65b9484d670f
8bab230b2ecd,0.173762,8bab230b2ecd
8e348601fefc,0.190529,8e348601fefc
8bd2fe400b89,0.249323,8bd2fe400b89


In [72]:
#If any probability is less than zero set it to zero
submission.deal_probability[submission.deal_probability<0]=0

/Users/parisamofakham/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/parisamofakham/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/parisamofakham/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exe

In [73]:
# define the order of columns
submission[['item_id','deal_probability']]

,item_id,deal_probability
item_id,,
6544e41a8817,6544e41a8817,0.053024
65b9484d670f,65b9484d670f,0.271194
8bab230b2ecd,8bab230b2ecd,0.173762
8e348601fefc,8e348601fefc,0.190529
8bd2fe400b89,8bd2fe400b89,0.249323
c63dbd6c657f,c63dbd6c657f,0.180449
6d1a410df86e,6d1a410df86e,0.108161
e8d3e7922b80,e8d3e7922b80,0.019931
2bc1ab208462,2bc1ab208462,0.091900


In [74]:
#Save the test prediction into a csv file
submission.to_csv('submission.csv', index=False)